<p>Usually for weight initializations, there are 3 points to watch out for:</p>
<ul>
    <ol>
        <li>Weights should not be the same.</li>
        <li>Weights should be small.</li>
        <li>Weights should have a good variance</li>
    </ol>
</ul>
<p>This is because every Neuron should be learning different aspects of the model</p>

In [1]:
import numpy as np # linear algebra 
import matplotlib.pyplot as plt # plotting 
import pandas as pd # dataframe

In [68]:
data = pd.read_csv("./Desktop/datasets/Churn_Modelling.csv")

In [69]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [70]:
len(data.columns)

# Exited is the dependent feature 
# other columns is the independent features

14

In [71]:
cat_columns = [key for key in data.columns if data[key].dtype == "object"]
num_columns = cat_columns = [key for key in data.columns if data[key].dtype != "object"]

In [72]:
cat_columns

['RowNumber',
 'CustomerId',
 'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [74]:
X = data.iloc[:, 3:13]
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [75]:
y = data.iloc[:, 13]
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [76]:
# Create dummy variables 
X["Geography"].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [77]:
geography = pd.get_dummies(X["Geography"], drop_first = True )
gender = pd.get_dummies(X["Gender"], drop_first = True)

In [78]:
geography

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,1,0


In [79]:
gender

,Male
0,0
1,0
2,0
3,0
4,0
...,...
9995,1
9996,1
9997,0
9998,1


In [80]:
X = pd.concat([X, geography, gender], axis = 1)

In [81]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


In [82]:
X.drop(columns = ["Geography", "Gender"], axis = 1, inplace = True)

In [83]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [84]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [85]:
X_test.shape # the rows we will use to test our model 

(2000, 11)

In [86]:
X_train.shape # the rows we use to fit our model 

(8000, 11)

In [87]:
y_train.shape # the labels on our train model

(8000,)

In [88]:
X_test

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
9394,597,35,8,131101.04,1,1,1,192852.67,1,0,0
898,523,40,2,102967.41,1,1,0,128702.10,0,0,0
2398,706,42,8,95386.82,1,1,1,75732.25,0,1,0
5906,788,32,4,112079.58,1,0,0,89368.59,0,0,1
2343,706,38,5,163034.82,2,1,1,135662.17,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1037,625,24,1,0.00,2,1,1,180969.55,0,0,0
2899,586,35,7,0.00,2,1,0,70760.69,0,0,0
9549,578,36,1,157267.95,2,1,0,141533.19,0,1,1
2740,650,34,4,142393.11,1,1,1,11276.48,1,0,1


In [36]:
y_test.shape # the labels on our test model 

(2000,)

In [93]:
y_train

7389    0
9275    0
2995    0
5316    0
356     0
       ..
9225    0
4859    0
3264    0
9845    0
2732    1
Name: Exited, Length: 8000, dtype: int64

RowNumber               2733
CustomerId          15592816
Surname             Udokamma
CreditScore              623
Geography            Germany
Gender                Female
Age                       48
Tenure                     1
Balance            108076.33
NumOfProducts              1
HasCrCard                  1
IsActiveMember             0
EstimatedSalary    118855.26
Exited                     1
Name: 2732, dtype: object

In [37]:
# Feature Scaling 
from sklearn.preprocessing import StandardScaler 
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

In [40]:
X_train[0]

array([ 0.16958176, -0.46460796,  0.00666099, -1.21571749,  0.8095029 ,
        0.64259497, -1.03227043,  1.10643166, -0.5698444 ,  1.74309049,
       -1.09168714])

In [42]:
import keras 
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout

In [43]:
classifier = Sequential()

In [45]:
# Adding the input and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = "he_uniform", activation = "relu", input_dim = 11))

In [46]:
classifier.add(Dense(units = 6, kernel_initializer = "he_uniform", activation = "relu"))

In [48]:
classifier.add(Dense(units = 1, kernel_initializer = "glorot_uniform", activation = "sigmoid"))

In [50]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [51]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [55]:
model_history = classifier.fit(X_train, y_train, validation_split = 0.33, batch_size = 10, epochs = 100)

Epoch 1/100


C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


536/536 [==============================] - 1s 879us/step - loss: 0.5872 - accuracy: 0.7509 - val_loss: 0.5125 - val_accuracy: 0.7891
Epoch 2/100
536/536 [==============================] - 0s 685us/step - loss: 0.4877 - accuracy: 0.7960 - val_loss: 0.4731 - val_accuracy: 0.8012
Epoch 3/100
536/536 [==============================] - 0s 778us/step - loss: 0.4540 - accuracy: 0.8018 - val_loss: 0.4530 - val_accuracy: 0.8054
Epoch 4/100
536/536 [==============================] - 0s 695us/step - loss: 0.4327 - accuracy: 0.8156 - val_loss: 0.4391 - val_accuracy: 0.8092
Epoch 5/100
536/536 [==============================] - 0s 724us/step - loss: 0.4162 - accuracy: 0.8240 - val_loss: 0.4248 - val_accuracy: 0.8137
Epoch 6/100
536/536 [==============================] - 0s 776us/step - loss: 0.3998 - accuracy: 0.8322 - val_loss: 0.4107 - val_accuracy: 0.8262
Epoch 7/100
536/536 [==============================] - 0s 654us/step - loss: 0.3851 - accuracy: 0.8410 - val_loss: 0.3981 - val_accuracy: 0.83

536/536 [==============================] - 0s 661us/step - loss: 0.3283 - accuracy: 0.8640 - val_loss: 0.3546 - val_accuracy: 0.8546
Epoch 58/100
536/536 [==============================] - 0s 659us/step - loss: 0.3278 - accuracy: 0.8651 - val_loss: 0.3549 - val_accuracy: 0.8573
Epoch 59/100
536/536 [==============================] - 0s 650us/step - loss: 0.3279 - accuracy: 0.8617 - val_loss: 0.3562 - val_accuracy: 0.8565
Epoch 60/100
536/536 [==============================] - 0s 654us/step - loss: 0.3283 - accuracy: 0.8653 - val_loss: 0.3538 - val_accuracy: 0.8576
Epoch 61/100
536/536 [==============================] - 0s 692us/step - loss: 0.3280 - accuracy: 0.8653 - val_loss: 0.3530 - val_accuracy: 0.8561
Epoch 62/100
536/536 [==============================] - 0s 654us/step - loss: 0.3284 - accuracy: 0.8653 - val_loss: 0.3534 - val_accuracy: 0.8557
Epoch 63/100
536/536 [==============================] - 0s 663us/step - loss: 0.3276 - accuracy: 0.8658 - val_loss: 0.3562 - val_accuracy

In [57]:
y_pred = classifier.predict(X_test)

63/63 [==============================] - 0s 354us/step


In [58]:
y_pred

array([[0.36077937],
       [0.3482305 ],
       [0.20470008],
       ...,
       [0.12207298],
       [0.15427488],
       [0.23389912]], dtype=float32)

In [60]:
y_pred = (y_pred > 0.5)

In [61]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [63]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

In [64]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_pred, y_test)

In [65]:
cm

array([[1502,   93],
       [ 188,  217]], dtype=int64)

In [66]:
score

0.8595

In [67]:
y_test

9394    0
898     1
2398    0
5906    0
2343    0
       ..
1037    0
2899    0
9549    0
2740    0
6690    0
Name: Exited, Length: 2000, dtype: int64